In [2]:
import os
import openai

import os
from dotenv import load_dotenv

load_dotenv()

openai.organization = "org-hCHXZmK4qtKP5ZQeCqtB9MV1"
openai.api_key = os.getenv("OPENAI_API_KEY")
#openai.Model.list()

import pandas as pd

In [38]:
pd.options.display.max_rows=500

In [3]:
# set up the prompt and model parameters
prompt = """Extraer fecha del evento, edad de la víctima, transporte del victimario, transporte de la víctima y ubicación del accidente:

AGUASCALIENTES, AGS.- Una mujer tuvo una muerte espantosa en el último día del 2018 tras de que fue atropellada y arrastrada varios metros por una camioneta. El trágico accidente sucedió el lunes 31 de diciembre, minutos antes de las siete de la noche, sobre la avenida Convención y la calle Pánfilo Natera, en el fraccionamiento Jesús Terán. La víctima mortal no había sido identificada, por lo que solamente se logró conocer que tenía entre 50 y 55 años de edad. El que la atropelló dijo llamarse Juan Ramón Calvillo Robledo, de 36 años y con domicilio en la avenida Paseo de la Cruz del fraccionamiento Torres de Ojocaliente, que conducía una camioneta Chevrolet Cheyenne, tipo Pick-Up, en color blanco y con placas de circulación AF-5561-A de esta entidad. Él transitaba por el Primer Anillo en dirección de sur a norte, sobre el carril izquierdo, cuando de pronto le salió al paso la peatón, que trataba de cruzar la vía. Sin embargo, antes de que llegara al camellón central fue brutalmente impactada por la parte frontal de la unidad, por lo que cayó al piso y enseguida la arrastró entre 30 y 40 metros. Juan Ramón, desde que vio a la transeúnte, aplicó los frenos, pero no logró detenerse y la arrolló. Una vez que finalmente se detuvo, el cuerpo de la fémina quedó a un lado de la unidad de motor. Testigos del accidente lo reportaron a los servicios de emergencia, por lo que al lugar acudieron paramédicos municipales a bordo de la ambulancia UE-58, que revisaron a la atropellada y confirmaron que ya había fallecido. También llegaron oficiales de la Policía Vial en la patrulla 986 para intervenir en el hecho, asegurando al conductor de la camioneta para presentarlo ante el Agente del Ministerio Público para deslindar responsabilidades. Finalmente, al sitio arribaron elementos de Servicios Periciales para levantar el cuerpo y trasladarlo al SEMEFO. 

								Funciona gracias a WordPress								
 | 
								Tema: Newsup de Themeansar"""
model = "text-davinci-003"

# make the request to the OpenAI API
response = openai.Completion.create(
  engine=model,
  prompt=prompt,
  max_tokens=1000,
  n=1,
  stop=None,
  temperature=0,
)



In [74]:
response.choices[0].text

'\n\nFecha del evento: lunes 31 de diciembre\nEdad de la víctima: entre 50 y 55 años\nTransporte del victimario: camioneta Chevrolet Cheyenne, tipo Pick-Up, en color blanco\nTransporte de la víctima: a pie\nUbicación del accidente: avenida Convención y la calle Pánfilo Natera, en el fraccionamiento Jesús Terán.'

In [21]:
df = pd.read_csv('output_data.csv', encoding='latin1')

df['texto_0'] = df.texto.copy()
df.loc[df.texto.isnull(), 'texto_0'] = ''

df['gpt_output'] = ''
df['Fecha del evento'] = ''
df['Edad de la víctima'] = ''
df['Transporte del victimario'] = ''
df['Transporte de la víctima'] = ''
df['Ubicación del accidente'] = ''

In [102]:
out_dict = output_to_dict(output)

In [179]:
def output_to_dict(output):

    output = output.split('\n')
    output = [x for x in output if x!='']
    
    if len(output)>0:
    
        dict_keys = []
        dict_values = []
        for x in output:
            
            if len(x.split(': '))==2:
                a, b = x.split(': ')
                dict_keys.append(a)
                dict_values.append(b)
            
        out_dict = dict(zip(dict_keys,dict_values))
        
        return out_dict

In [126]:
%%time

for i in df[ 
    ~df.texto_0.str.contains('weve detected|requested url|requested resource|access the url',case=False) & 
    ~df.texto_0.str.contains('error 404',case=False) & 
    df.texto.notnull()
    ].head(10).index:
    
    
    # set up the prompt and model parameters
    prompt = "Extraer fecha del evento, edad de la víctima, transporte del victimario, transporte de la víctima y ubicación del accidente:\n "
    prompt += df.loc[i, 'texto']
    model = "text-davinci-003"
    
    try:
        # make the request to the OpenAI API
        response = openai.Completion.create(
          engine=model,
          prompt=prompt,
          max_tokens=1900,
          n=1,
          stop=None,
          temperature=0,
        )
        output = response.choices[0].text

    except Exception as error:
        print('Caught this error: ' + repr(error))
        output = ''
    
    df.loc[i, 'gpt_output'] = output
    
    out_dict = output_to_dict(output)
    
    if out_dict!=None:
    
        for key, value in out_dict.items():
            
            if key in df.columns:
                df.loc[i, key] = value
            
    print(i)

0
1
2
5
8
9
12
13
15
Caught this error: InvalidRequestError(message="This model's maximum context length is 4097 tokens, however you requested 4421 tokens (2521 in your prompt; 1900 for the completion). Please reduce your prompt; or completion length.", param=None, code=None, http_status=400, request_id=None)
17
CPU times: total: 422 ms
Wall time: 37 s


In [184]:
df

,URL noticia,sitio,milenio_dummy,elsol_dummy,texto,tags,status_code,edad,modo,texto_0,gpt_output,Fecha del evento,Edad de la víctima,Transporte del victimario,Transporte de la víctima,Ubicación del accidente
0,http://adnsureste.info/muere-persona-atropella...,adnsureste.info,0,0,Tu dirección de correo electrónico no será pub...,NaN,NaN,NaN,peaton,Tu dirección de correo electrónico no será pub...,\nFecha del evento: 15 de mayo de 2020\nEdad d...,15 de mayo de 2020,18 años,Automóvil,Autobús,Calle principal de la ciudad.
1,https://noticieroelcirco.com/mujer-murio-atrop...,noticieroelcirco.com,0,0,"AGUASCALIENTES, AGS.- Una mujer tuvo una muert...",NaN,NaN,55,peaton,"AGUASCALIENTES, AGS.- Una mujer tuvo una muert...",\n\nFecha del evento: lunes 31 de diciembre\nE...,lunes 31 de diciembre,entre 50 y 55 años,"camioneta Chevrolet Cheyenne, tipo Pick-Up, en...",a pie,"avenida Convención y la calle Pánfilo Natera, ..."
2,https://www.siglo.mx/coahuila/noticia/256815.m...,www.siglo.mx,0,0,\n\n RENÉ ARELLANO/ EL ...,NaN,NaN,tres,peaton,\n\n RENÉ ARELLANO/ EL ...,\n\nFecha del evento: martes 01 de enero 2019\...,martes 01 de enero 2019,3 años,"Tracto camión marca Kenworth, color blanco con...",No especificado,"Colonia San Joaquín, Piedras Negras, Coahuila."
3,https://www.facebook.com/Red933MX/videos/36355...,www.facebook.com,0,0,NaN,NaN,NaN,NaN,NaN,,,,,,,
4,https://novedadesaca.mx/muere-en-hospital-muje...,novedadesaca.mx,0,0,NaN,NaN,NaN,NaN,NaN,,,,,,,
5,https://www.razon.com.mx/mexico/incendian-patr...,www.razon.com.mx,0,0,Copyright © La Razón Todos los derechos rese...,NaN,NaN,NaN,peaton,Copyright © La Razón Todos los derechos rese...,\nFecha del evento: 28 de mayo de 2020\nEdad d...,28 de mayo de 2020,18 años,Automóvil,Bicicleta,Carretera federal México-Puebla
6,http://binoticias.com/nota.cfm?id=44775&t=ya-f...,binoticias.com,0,0,NaN,NaN,NaN,NaN,NaN,,,,,,,
7,https://atnoticias.com.mx/notas_Muri%C3%B3_al_...,atnoticias.com.mx,0,0,NaN,NaN,1.0,NaN,NaN,,,,,,,
8,https://www.diariocambio.com.mx/2018/regiones/...,www.diariocambio.com.mx,0,0,La mañana de este miércoles fue reportada la p...,NaN,NaN,NaN,peaton,La mañana de este miércoles fue reportada la p...,\n\nFecha del evento: Miércoles\nEdad de la ví...,Miércoles,Desconocida,Desconocido,Desconocido,Boulevard Arco Poniente a la altura del kilóme...
9,https://www.elsoldedurango.com.mx/local/estrem...,www.elsoldedurango.com.mx,0,1,Estremecedor: muere atropellado niño de 2 años...,"policiaca, durango, arrollado",200.0,2,peaton,Estremecedor: muere atropellado niño de 2 años...,\n\nFecha del evento: Miércoles \nEdad de la v...,Miércoles,2 años,"Camioneta Ford, tipo Pick-Up, modelo 1979, col...",No se especifica,Calle 7 Leguas del fraccionamiento Valle del M...


In [190]:
df.to_csv('gpt_output_data.csv', index=False, encoding='latin1', errors='ignore')

## catch extraction error texts

In [42]:
aux = df[ ~df.texto_0.str.contains('weve detected|requested url|requested resource|access the url',case=False) & 
   ~df.texto_0.str.contains('error 404',case=False) & 
   df.texto.notnull()].groupby('texto').sitio.count().reset_index().sort_values('sitio',ascending=False)

In [44]:
aux

,texto,sitio
59,An appropriate representation of the requested...,18
229,The requested URL was not found on this server...,9
288,Â© Copyright AM 2023. Todos los derechos reser...,7
53,14 Guanajuato © Copyright Periódico Correo 20...,6
228,The requested URL was not found on this server.,5
138,Los mochis Culiacan Mazatlan El fuerte Los moc...,4
0,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSáb0...,4
187,"Para tener la información al momento, suscríbe...",3
217,Se lleva al cabo el entierro de la adolescente...,2
137,Los cuerpos quedaron a orilla de la carretera ...,2


In [165]:
df[ ~df.texto_0.str.contains('weve detected',case=False) & 
   ~df.texto_0.str.contains('error 404',case=False) & 
   df.texto.notnull()]

,URL noticia,sitio,milenio_dummy,elsol_dummy,texto,tags,status_code,edad,modo,texto_0,gpt_output,Fecha del evento,Edad de la víctima,Transporte del victimario,Transporte de la víctima,Ubicación del accidente
0,http://adnsureste.info/muere-persona-atropella...,adnsureste.info,0,0,Tu dirección de correo electrónico no será pub...,NaN,NaN,NaN,peaton,Tu dirección de correo electrónico no será pub...,\nFecha del evento: 15 de mayo de 2020\nEdad d...,No especificada,No especificada,Tráileres,Autobús de pasajeros,Carretera Mérida a Cancún
1,https://noticieroelcirco.com/mujer-murio-atrop...,noticieroelcirco.com,0,0,"AGUASCALIENTES, AGS.- Una mujer tuvo una muert...",NaN,NaN,55,peaton,"AGUASCALIENTES, AGS.- Una mujer tuvo una muert...",\n\nFecha del evento: lunes 31 de diciembre\nE...,No especificada,No especificada,Tráileres,Autobús de pasajeros,Carretera Mérida a Cancún
2,https://www.siglo.mx/coahuila/noticia/256815.m...,www.siglo.mx,0,0,\n\n RENÉ ARELLANO/ EL ...,NaN,NaN,tres,peaton,\n\n RENÉ ARELLANO/ EL ...,\n\nFecha del evento: martes 01 de enero 2019\...,No especificada,No especificada,Tráileres,Autobús de pasajeros,Carretera Mérida a Cancún
5,https://www.razon.com.mx/mexico/incendian-patr...,www.razon.com.mx,0,0,Copyright © La Razón Todos los derechos rese...,NaN,NaN,NaN,peaton,Copyright © La Razón Todos los derechos rese...,\nFecha del evento: 28 de mayo de 2020\nEdad d...,No especificada,No especificada,Tráileres,Autobús de pasajeros,Carretera Mérida a Cancún
8,https://www.diariocambio.com.mx/2018/regiones/...,www.diariocambio.com.mx,0,0,La mañana de este miércoles fue reportada la p...,NaN,NaN,NaN,peaton,La mañana de este miércoles fue reportada la p...,\n\nFecha del evento: Miércoles\nEdad de la ví...,No especificada,No especificada,Tráileres,Autobús de pasajeros,Carretera Mérida a Cancún
9,https://www.elsoldedurango.com.mx/local/estrem...,www.elsoldedurango.com.mx,0,1,Estremecedor: muere atropellado niño de 2 años...,"policiaca, durango, arrollado",200.0,2,peaton,Estremecedor: muere atropellado niño de 2 años...,\n\nFecha del evento: Miércoles \nEdad de la v...,No especificada,No especificada,Tráileres,Autobús de pasajeros,Carretera Mérida a Cancún
12,https://www.tabascohoy.com/nota/465790/barrend...,www.tabascohoy.com,0,0,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSáb0...,NaN,NaN,5,peaton,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSáb0...,"\n\nFecha del evento: Sáb,04 Feb 2023\nEdad de...",No especificada,No especificada,Tráileres,Autobús de pasajeros,Carretera Mérida a Cancún
13,https://www.novaradio.mx/muere-un-hombre-atrop...,www.novaradio.mx,0,0,"Lo sentimos, página no encontrada Culiacán, Si...",NaN,NaN,NaN,peaton,"Lo sentimos, página no encontrada Culiacán, Si...",\n\nFecha del evento: 3 de febrero del 2023\nE...,No especificada,No especificada,Tráileres,Autobús de pasajeros,Carretera Mérida a Cancún
15,https://www.yucatan.com.mx/merida/policia/un-h...,www.yucatan.com.mx,0,0,En la carretera MÃ©rida a CancÃºn se registrÃ³...,NaN,NaN,NaN,peaton,En la carretera MÃ©rida a CancÃºn se registrÃ³...,\n\nFecha del evento: No especificada\nEdad de...,No especificada,No especificada,Tráileres,Autobús de pasajeros,Carretera Mérida a Cancún
17,https://www.vanguardiaveracruz.mx/atropellado-...,www.vanguardiaveracruz.mx,0,0,Publicado - Por Un hombre cruzó la vía a la al...,NaN,NaN,NaN,moto,Publicado - Por Un hombre cruzó la vía a la al...,,No especificada,No especificada,Tráileres,Autobús de pasajeros,Carretera Mérida a Cancún
